# Frame by Frame

Este notebook tiene una entrada que sirve para analizar un video cuadro a cuadro. Al ejecutar dicha entrada se va a abrir una ventana donde va a aparecer el primer cuadro del video. Apretando la letra "n" se mostrará el proximo cuadro. Apretando "p" se mostrará el cuadro anterior. Si se aprieta la letra "t" mostrará el tiempo en segundos del cuadro. Si se hace click en alguna parte de la ventana mostrará la coordenada en pixeles donde se hizo click en el video y el tiempo del cuadro. Estos datos se guardan en un dataframe que se llama "df". Para cerrar el programa apretar la letra "q".

Al final del notebook hay una entrada que sirve para descargar videos de youtube. Usé esto para descargar un video de una persona caminando.

In [3]:
import cv2
import pandas as pd
from glob import glob as gl

In [4]:
gl('*')

['frame by frame2.ipynb',
 'README.md',
 'tiro_oblicuo.csv',
 'video.mp4',
 'video2.mp4']

In [5]:
def get_video_info(video_path):
    video = cv2.VideoCapture(video_path)
    frame_count = int(video.get(cv2.CAP_PROP_FRAME_COUNT))
    fps = video.get(cv2.CAP_PROP_FPS)
    duration = frame_count / fps
    return frame_count, fps, duration

def format_time(seconds):
    minutes, seconds = divmod(seconds, 60)
    hours, minutes = divmod(minutes, 60)
    return "{:02d}:{:02d}:{:02d}".format(int(hours), int(minutes), int(seconds))

def display_frame(video, frame_number):
    video.set(cv2.CAP_PROP_POS_FRAMES, frame_number)
    _, frame = video.read()
    global width
    global height
    width = 800 # Aca pongan el ancho del video que quieren ver en pantalla
    height = 600 # Aca el alto
    frame = cv2.resize(frame, (width ,height))
    
    cv2.imshow('Video Player', frame)
    frame_count = video.get(cv2.CAP_PROP_POS_FRAMES)
    duration = video.get(cv2.CAP_PROP_POS_MSEC) / 1000
    #print("Frame: {}".format(int(frame_count)))
    #print("Time: {}".format(format_time(duration)))
    #print("Time: {}".format(duration))
    return(duration)

def showtime(video, frame_number):
    video.set(cv2.CAP_PROP_POS_FRAMES, frame_number)
    frame_count = video.get(cv2.CAP_PROP_POS_FRAMES)
    duration = video.get(cv2.CAP_PROP_POS_MSEC) #/1000
    #print('new_time',frame_count/30)
    print('time',duration)
    return(duration)

def mouse_callback(event, x, y, flags, param):
    if event == cv2.EVENT_LBUTTONDOWN:
        video = cv2.VideoCapture(video_path)
        
        frame_count = video.get(cv2.CAP_PROP_POS_FRAMES)
        
        print(f"Mouse clicked at ({x}, {y})")
        t = showtime(video,current_frame)
        global df
        df = pd.concat([df,pd.DataFrame({'t':[t],
                                         'frame':[current_frame],
                                       'x':[x],
                                       'y':[height-y]})],ignore_index = True)
        df['t_frames'] = df.frame/30
       
def main():
    global video_path
    #video_path = input("Enter the path to the video file: ")
    video_path = 'video2.mp4' #bre si esta en la misma carpeta que este notebook
    video = cv2.VideoCapture(video_path)
    
    if not video.isOpened():
        print("Failed to open video file.")
        return

    total_frames, fps, duration = get_video_info(video_path)
    #print("Total Frames: {}".format(total_frames))
    #print("FPS: {}".format(fps))
    #print("Duration: {} seconds".format(int(duration)))

    global current_frame
    current_frame = 0
    time = display_frame(video, current_frame)
    cv2.setMouseCallback('Video Player', mouse_callback)
    while True:
        key = cv2.waitKey(0)

        if key == ord('q'):
            break
        elif key == ord('n'):
            current_frame += 1
            if current_frame >= total_frames:
                current_frame = total_frames - 1
            display_frame(video, current_frame)
        elif key == ord('p'):
            current_frame -= 1
            if current_frame < 0:
                current_frame = 0
            display_frame(video, current_frame)
        elif key == ord('t'):
            showtime(video, current_frame)
            #print("Time: {}".format(time))

    video.release()
    cv2.destroyAllWindows()

global df
df = pd.DataFrame()
if __name__ == '__main__':
    main()


Mouse clicked at (29, 368)
time 0.044444444444444446


In [5]:
df

,t,frame,x,y,t_frames
0,2.511111,226,30,232,7.533333
1,2.522222,227,35,242,7.566667
2,2.533333,228,44,254,7.600000
3,2.544444,229,53,266,7.633333
4,2.555556,230,60,275,7.666667
...,...,...,...,...,...
82,3.455556,311,517,352,10.366667
83,3.466667,312,523,346,10.400000
84,3.477778,313,529,338,10.433333
85,3.488889,314,533,331,10.466667


## df


In [6]:
#H:/Mi unidad/Tesis-codigo/datos crudos/Sujeto 4 Datos/S04-C-F-S25.avi
df.to_csv ('tiro_oblicuo.csv',index = False)

## Graficar

In [7]:
import plotly.graph_objects as go


In [9]:
fig = go.Figure()
fig.add_traces(go.Scatter( x = df.x, y=df.y))

In [15]:
def estetica(figura,x_name = '',y_name = '',title = '',w = 400,h = 325):
    
    figura.update_xaxes(showgrid=False,showline=True, linewidth=2, linecolor='black', mirror=True,title = x_name)#'cccccccccccccccc')
    figura.update_yaxes(showgrid=False,showline=True, linewidth=2, linecolor='black', mirror=True,title =y_name)


    figura.update_layout(
        margin=dict(l=20, r=20, t=40, b=20),
        plot_bgcolor="white",
        template="plotly_white", 
        title=title,
        autosize=False,
        width=w,
        height=h,
        #yaxis = dict(range=[0,1]),
        #xaxis = dict(range=[0,1]),
        title_font = dict(size =20),
        legend=dict(orientation = 'v',
                    font = dict(size = 18),
                     yanchor="top",
                     y=0.95,
                     xanchor="left",
                     x=0.05,
                     bgcolor='rgba(255,255,255,0)',
                   )
    )

In [16]:
fig = go.Figure()
fig.add_traces(go.Scatter(x = df.t,y = df.x, name = 'X'))
fig.add_traces(go.Scatter(x = df.t,y = df.y, name = 'Y'))
estetica(fig,title = 'Titulo de la figura',x_name = 'Tiempo (seg)', y_name = 'Pixeles', h = 500,w = 600)
fig.show()

In [90]:
df['vx'] = df.x.diff(5)/df.t.diff(5)
df['vy'] = df.y.diff(5)/df.t.diff(5)

In [93]:
fig = go.Figure()
fig.add_traces(go.Scatter(x = df.t,y = df.x,
                          name = 'X',
                          hovertemplate ='<b>Vx</b>: %{text}',
                          text = df.vx,))

fig.add_traces(go.Scatter(x = df.t,y = df.y, 
                          name = 'Y', 
                          hovertemplate ='<b>Vy</b>: %{text}',
                          text = df.vy,))

estetica(fig,title = 'Posición',x_name = 'Tiempo (seg)', y_name = 'Pixeles', h = 500,w = 600)

fig.show()

fig2 = go.Figure()
fig2.add_traces(go.Scatter(x = df.t,y = df.vx,
                          name = 'X',
                          hovertemplate ='<b>Vx</b>: %{text}',
                          text = df.vx,))

fig2.add_traces(go.Scatter(x = df.t,y = df.vy, 
                          name = 'Y', 
                          hovertemplate ='<b>Vy</b>: %{text}',
                          text = df.vy,))

estetica(fig2,title = 'Velocidad',x_name = 'Tiempo (seg)', y_name = 'Pixeles/seg', h = 500,w = 600)
fig2.update_layout(
        legend=dict(orientation = 'v',
                    font = dict(size = 18),
                     yanchor="top",
                     y=0.95,
                     xanchor="right",
                     x=0.95,
                     bgcolor='rgba(255,255,255,0)',
                   ))

fig2.show()

## Filtrado de señal

In [77]:
from scipy.signal import savgol_filter as sv_filt
from scipy import signal

In [88]:
tot_points = len(df)
d = 5
df['x_filt'] = sv_filt(df.x,15,3)
df['y_filt'] = sv_filt(df.y,15,3)
df['vx2'] = df.x_filt.diff(d)/df.t.diff(d)
df['vy2'] = df.y_filt.diff(d)/df.t.diff(d)

In [89]:
fig = go.Figure()
fig.add_traces(go.Scatter(x = df.t,y = df.x_filt,
                          name = 'X',
                          hovertemplate ='<b>Vx</b>: %{text}',
                          text = df.vx2,))

fig.add_traces(go.Scatter(x = df.t,y = df.y_filt, 
                          name = 'Y', 
                          hovertemplate ='<b>Vy</b>: %{text}',
                          text = df.vy2,))

estetica(fig,title = 'Titulo de la figura',x_name = 'Tiempo (seg)', y_name = 'Pixeles', h = 500,w = 600)

fig.show()